# Project 2: Named Entity Recognition (NER) with Sequence Labeling Models
## CS4740/5740 Fall 2021

### Project Submission Due: Oct 15th, 2021 (11.59PM)
Please submit **pdf file** of this notebook on **Gradescope**, and **ipynb** on **CMS**. For instructions on generating pdf and ipynb files, please refer to project 1 instructions.



**Names: Lusca Robinson, Kyrus Mama**

**Netids: nar73, krm74**

Don't forget to share your newly copied notebook with your partner!


**Reminder: both of you can't work in this notebook at the same time from different computers/browser windows because of sync issues. We even suggest to close the tab with this notebook when you are not working on it so your partner doesn't get sync issues.**


# **Introduction** 🔎

---

In this project, you will implement a model that identifies named entities in text and tags them with the appropriate label. Particularly, the task of this project is **Named Entity Recognition**. A primer on this task is provided further on. The given dataset is a modified version of the CoNLL-2003 ([Sang et al](https://arxiv.org/pdf/cs/0306050v1.pdf)) dataset. Please use the datasets that we have released to you instead of versions found online as we have made simplifications to the dataset for your benefit. Your task is to develop NLP models to identify these named entities automatically. We will treat this as a **sequence-tagging task**: for each token in the input text, assign one of the following 5 labels: **ORG** (Organization), **PER** (Person), **LOC** (Location), **MISC** (Miscellaneous), and **O** (Not Named Entity). More information about the dataset is provided later

For this project, you will implement two sequence labeling approaches:
- Model 1 : a Hidden Markov Model (HMM)
- Model 2 : a Maximum Entropy Markov Model (MEMM), which is an adaptation of an HMM in which a Logistic Regression classifier (also known as a MaxEnt classifier) is used to obtain the lexical generation probabilities (i.e., the observation/emission probability matrix, so "observations" == "emissions" == "lexical generations"). Feature engineering is strongly suggested for this model!

Implementation of the Viterbi algorithm (for finding the most likely tag sequence to assign to an input text) is required for both models above, so make sure that you understand it ASAP.

You will implement and train two sequence tagging models, generate your predictions for the provided test set, and submit them to **Kaggle**. Please enter all code in this colab notebook and answer all the questions in the supporting document.

To refresh your memory on HMMs, MEMMs, and Viterbi you can refer to **Jurafsky & Martin Ch. 8.3–8.5** and the lecture slides which can be found on EdStem.

## **Logistics**

---

- You **must** work in **groups of 2 students**. Students in the same group will get the same grade. Thus, you should make sure that everyone in your group contributes to the project. 
- **Remember to form groups on BOTH CMS and Gradescope** or not all group members will receive grades. You can use make a post on EdStem to find a partner for this project.
- Please complete the written questions of this notebook in a clear and informative way. We have created a template document for you to answer the written questions. This document can be found [here](https://docs.google.com/document/d/1vnxYFS-rxxLOYfKG6YN35YktJZnzqQBhE1Mj1Xu7IF0/edit?usp=sharing). Please make a copy of this document for yourself and add your names and netids in the header and answer the written questions on it. You will need to submit this document to gradescope as well (do not forget to do this please!).
- At the end: please make sure to submit the following 3 items:
  1. PDF version of Colab notebook on Gradescope (instructions for converting to PDF are at the end).
  2. PDF version of Google Doc with written answers to the numbered questions on this colab on Gradescope.
  3. .ipynb version of your colab notebook on CMS.

- Note: When submitting the PDF documents to Gradescope (colab notebook & writeup doc) please join/concatenate the PDFs and then submit them as one. You may do this any way you please. You can use [this](https://pdfjoiner.com/) website if you wish to.

## **Advice**

---

1. Please read through the entire notebook before you start coding. That might inform your code structure.
2. Grading breakdown is found at the end; please consult it.
3. Google colab does **not** provide good synchronization; we do not recommend multiple people to work on the same notebook at the same time.
4. The project is somewhat open ended. ("But that's a good thing.  Really. It's more fun that way", says Claire.) We will ask you to implement some model, but precise data structures and so on can be chosen by you. However, to integrate with Kaggle, you will need to submit Kaggle predictions using the given evaluation code (more instructions later).
5. You will be asked to fill in your code at various points of the document. You will also be asked to answer questions that analyze your results and motivate your implementation. Please answer these on an additional writeup document. A template has been provided to you.

## **Named Entity Recognition: A Primer**

---

Let us now take a look at the task at hand: Named Entity Recognition (NER). This section provides a brief introduction to the task and why it is important.

**What is NER?**
NER refers to the information extraction technique of identifying and categorizing key information about entities within textual data. Let's look at an example: 

<br/>

![picture](https://drive.google.com/uc?id=1mxwn1_2Ef16_MJeyl9jJwwR6IohUOeHO)

<br/>

In the above example, we can see that the text has numerous named entities that can be categorized as LOC (location), ORG (organization), PER (person), etc. Today, the task of NER has been overwhelmed by deep learning approaches. However, for this assignment, we will try to do NER using something simpler: HMMs and MEMMs. NER is important for a number of reasons and has a wide variety of use cases such as but not limited to:
  - Detect entities in search engines and voice assistants for more relavent search results.
  - Automatically parsing resumes.
  - ...and many more!


To read more on NER, we refer to any of the following sources:
1. Medium post [1](https://umagunturi789.medium.com/everything-you-need-to-know-about-named-entity-recognition-2a136f38c08f) and [2](https://medium.com/mysuperai/what-is-named-entity-recognition-ner-and-how-can-i-use-it-2b68cf6f545d).
2. Try out [this](https://demo.allennlp.org/named-entity-recognition/named-entity-recognition) AlllenNLP demo!

## **Entity Level Mean F1**

---

Let's take a look at the metrics that you will focus on in this assignment. The standard measures to report for NER are recall, precision, and F1 score
(also called F-measure) evaluated at the **named entity level** (not at the token level). The code for this has been provided later under the validation section under Part 2. Please use this code when evaluating your models. 


If P and T are the sets of predicted and true *named entity spans*, respectively, (e.g, the five named entity spans in the above example are "Zifa", "Renate Goetschl", "Austria", "World Cup", and "Germany") then

####<center>Precision = $\frac{|\text{P}\;\cap\;\text{C}|}{|\text{C}|}$ and Recall = $\frac{|\text{P}\;\cap\;\text{C}|}{|\text{P}|}$.</center><br/>


####<center>F1 = $\frac{2 * \text{Precision} * \text{Recall}}{\text{Precision} + \text{Recall}}$. </center><br/>

For each type of named entity, e.g. *LOC*ation, *MISC*ellaneous, *ORG*anization and *PER*son, we calculate the F1 score as shown above, and take the mean of all these F1 scores to get the **Entity Level Mean F1** score for the test set. If $N$ is the total number of labels (i.e., named entity types), then

####<center>Entity Level Mean F1 = $\frac{\sum_{i = 1}^{N} \text{F1}_{{label}_i}}{N}$. </center>

More details under the validation section in Part 2.



# **Part 1: Dataset** 📈

Load the dataset as follows:
  1. Obtain the data from Kaggle at https://www.kaggle.com/c/cs4740-fa21-p2/data.
  2. Unzip the data. Put it into your google drive, and mount it on colab as per below:

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import json
import math

# TODO: please change the line below with your drive organization
path = os.path.join(os.getcwd(), "drive", "MyDrive", "CS 4740", "Project 2", "Dataset")

with open(os.path.join(path,'train.json'), 'r') as f:
     train = json.loads(f.read())

with open(os.path.join(path,'test.json'), 'r') as f:
     test = json.loads(f.read())

Here's a few things to note about the dataset above:
1. We have just loaded 2 json files: train and test. Please note that these files are different from the original release of the CoNNL-2003 since we have already processed and tokenized them for you. Hence, the documents are represented as a list of strings. Note that it is **not** split into separate training and development/validation sets. You will need to do this yourself as needed using the train set.
2. The train file contains the following 4 fields (each is a nested list): 
  - **'text'** - actual input tokens
  - **'NER'** - the token-level entity tag (ORG/PER/LOC/MISC/O) where **O is used to denote tokens that are not part of any named entity**
  - **'POS'** - the part of speech tag (will be handy for feature engineering of the MEMM model)
  - **'index'** - index of the token in the dataset
3. The test data only has 'text', 'POS' and 'index' fields. You will need to submit your prediction of the 'NER' tag to Kaggle. More instructions on this later!

Let's take a look at a sample sentence from the dataset!

In [ ]:
print(train['text'][1])
print(train['index'][1])
print(train['POS'][1])
print(train['NER'][1])

lst = []
NER_counts = {}
for i in range(len(train['NER'])):
  for j in range(len(train['NER'][i])):
    if train['NER'][i][j] not in NER_counts:
      NER_counts[train['NER'][i][j]] = 1
    else:
      NER_counts[train['NER'][i][j]] += 1
print(NER_counts)

for i in range(len(train['text'])):
  lst.append(len(train['text'][i]))
  # print(len(train['text'][i]))
print("mean", sum(lst)/len(lst))
all_NER_list =[] 
for i in range(len(train['NER'])):
  all_NER_list.extend(train['NER'][i])
Oentries=0
for x in all_NER_list:
  if x == "O":
    Oentries += 1
print(Oentries/len(all_NER_list))


['Polish', 'schoolgirl', 'blackmailer', 'wanted', 'textbooks', '.', 'GDANSK', ',', 'Poland', '1996-08-22', 'A', 'Polish', 'schoolgirl', 'blackmailed', 'two', 'women', 'with', 'anonymous', 'letters', 'threatening', 'death', 'and', 'later', 'explained', 'that', 'she', 'needed', 'money', 'for', 'textbooks', ',', 'police', 'said', 'on', 'Thursday', '.', '"', 'The', '13-year-old', 'girl', 'tried', 'to', 'extract', '60', 'and', '70', 'zlotys', '(', '$', '22', 'and', '$', '26', ')', 'from', 'two', 'residents', 'of', 'Sierakowice', 'by', 'threatening', 'to', 'take', 'their', 'lives', ',', '"', 'a', 'police', 'spokesman', 'said', 'in', 'the', 'nearby', 'northern', 'city', 'of', 'Gdansk', 'on', 'Thursday', '.', 'He', 'said', 'the', 'women', 'reported', 'the', 'blackmail', 'letters', 'and', 'police', 'caught', 'the', 'girl', 'on', 'Wednesday', 'as', 'she', 'tried', 'to', 'pick', 'up', 'the', 'cash', 'at', 'the', 'Sierakowice', 'railway', 'station', '.', '"', 'Interviewed', 'in', 'the', 'presence'

As you can see, the above the sentence, "Romania state budget soars in June.", has already been tokenized into an array of word tokens. The index array corresponds to the index of the token in the entire dataset (not the sentence). The POS tags and the NER tags correspond to the given indices. For example, the token: **Romania** has:
  - index: 0
  - POS: 'NNP'
  - NER: **'ORG'**

### **Q1: Initial Data Observations**
What are your initial observations after you explore the dataset?  Provide some quantitative data exploration. Assess dataset size, document lengths and the token-level NER class distribution, and the entity-level NER class distribution (skipping the 'O' label for the latter). Give some examples of sentences with their named entities bracketed, e.g. [[LOC Romania] state budget soars in June .] and [[ORG Zifa] said [PER Renate Goetschl] of [LOC Austria]...]. 

Present your findings in the supporting template document!

# **Part 2: Hidden Markov Model** 🧨

---

1. Code for counting and smoothing of labels and words and unkown word handing as necessary to support the Viterbi algorithm. (This is pretty much what you already know how to do from project 1.)
2. Build a Hidden Markov Model in accordance with the starter code that has been provided. If you wish to change this starter code you can. However, please ensure that your code is clear, concise, and, most important of all, modular. So break your implementation down into functions or write it within a class. We suggest you compute all probabilities in a log form when building the HMM.
3. An implementation of the **Viterbi algorithm** that can be used to infer token-level labels (identifying the appropriate named entity) for an input document. This process is commonly referred to as **decoding**. Bigram-based Viterbi is $ \mathcal{O}(sm^2)$ where s is the length of the sentence and m is the number of tags. Your implementation should have similar efficiency. The code for this can be used later on for the MEMM too.

Code of Academic Integrity:  We encourage collaboration regarding ideas, etc. However, please **do not copy code from online or share code with other students**. We will be running programs to detect plagiarism.


## **Unknown Word Handling**
---

In [ ]:
# Implement unknown word handling here! You may do this any way that you please

UNKNOWN_TOKEN = "<UNK>"

def unknown_handling(tokens):
  return_tokens = []
  known_words = {}
  for i in range(len(tokens)):
    sentence = []
    for j in range(len(tokens[i])):
      token = tokens[i][j]
      if token not in known_words:
        sentence.append(UNKNOWN_TOKEN)
        known_words[token] = 0
      else:
        sentence.append(token)
    return_tokens.append(sentence)
    
  return return_tokens


## **HMM Implementation**

---

The code below is just a suggestion for how you may go about building your HMM. Feel free to change it any way you see fit. In fact, you will probably have to tweak it a little to implement smoothing. In the skeleton code below, we have broken down the HMM into its three components: the transition matrix, the emission (i.e., lexical generation, observation) matrix, and the starting state probabilities. We suggest you implement them separately and then use them to build the HMM.

Note: it may help to map your classes (named entity types) to discrete values rather than string labels as you will have to do this for your MEMM anyways. However, the HMM can be done without this.

In [ ]:
# returns the transition probabilities
def build_transition_matrix(labels, k=1):
  transition = {}
  for i in range(len(labels)):
    for j in range(len(labels[i])-1):
      prev = labels[i][j]
      post = labels[i][j+1]
      if prev not in transition:
        transition[prev] = {}

      if post not in transition[prev]:
        transition[prev][post] = 1
      else:
        transition[prev][post] += 1

  classes = list(transition.keys())

  for prev in transition.keys():
    total = 0
    for post in transition[prev].keys():
      total += transition[prev][post]
    for post in transition[prev].keys():
      # transition[prev][post] /= total
      transition[prev][post] =  (transition[prev][post] + k) / (total + k * len(classes))

    for pot_post in classes:
      if pot_post not in transition[prev]:
        transition[prev][pot_post] = k / (total + k * len(classes))

  return transition

# print(build_transition_matrix(train['NER']))

In [ ]:
# returns the emission probabilities
def build_emission_matrix(tokens, labels, k=0.0001):
  emission = {}
  for i in range(len(labels)):
    for j in range(len(labels[i])):
      label = labels[i][j]
      token = tokens[i][j]

      if label not in emission:
        emission[label] = {}

      if token not in emission[label]:
        emission[label][token] = 1
      else:
        emission[label][token] += 1
  
  classes = list(emission.keys())
  for label in classes:
    for i in range(len(labels)):
      for j in range(len(labels[i])): 
        token = tokens[i][j]
        if token not in emission[label]:
          emission[label][token] = 0

  # vocab of all words in training
  V = len(list(emission[classes[0]].keys()))

  for label in emission.keys():
    total = 0
    for token in emission[label].keys():
      total += emission[label][token]
    for token in emission[label].keys():
      # emission[label][token] /= total
      emission[label][token] = (emission[label][token] + k) / (total + k * V)
  return emission 
  
# print(build_emission_matrix(train['text'], train['NER']))

In [ ]:
# returns the starting state probabilities
def get_start_state_probs(labels):
  initial = {}
  for i in range(len(labels)):
    label = labels[i][0]

    if label not in initial:
      initial[label] = 1
    else:
      initial[label] += 1

  total = 0
  for key in initial:
    total += initial[key]
  for key in initial:
    initial[key] /= total

  return initial

# print(get_start_state_probs(train['NER']))

In [ ]:
# takes in the tokens & labels and returns a representation of the HMM
# call the three functions above in this function to build your HMM
def build_hmm(tokens, labels):
  HMM = {}
  HMM['initial'] = get_start_state_probs(labels)
  HMM['transition'] = build_transition_matrix(labels)
  HMM['emission'] = build_emission_matrix(unknown_handling(tokens), labels)
  return HMM

# print(build_hmm(train['text'], train['NER']))

## **Viterbi Implementation**

---

At the end of your implementation, we expect a function or class that maps a sequence of tokens (observation) to a sequence of labels via the Viterbi algorithm.

In [ ]:
def get_emission_smoothing(emission_matrix, label, token):
  if token in emission_matrix[label]:
    return math.log(emission_matrix[label][token])

  # For words that exist somewhere in the emission matrix, if they dont occur 
  # for the current label we return a very negative number, considering 
  # those outcomes very unlikely
  # for possible_label in emission_matrix.keys():
  #   if token in emission_matrix[possible_label]:
  #     return -10000000 

  # if a word doesnt appear in the emission matrix at all, we skip that word's
  # emission probabilities for all labels.
  # print(label, token, emission_matrix[label][UNKNOWN_TOKEN])

  return math.log(emission_matrix[label][UNKNOWN_TOKEN])


def get_transition_smoothing(transition_matrix, pre, post):
  if pre in transition_matrix:
    if post in transition_matrix[pre]:
      return math.log(transition_matrix[pre][post])
  assert False


# takes in the hmm build above and an observation: list of tokens
# and returns the appropriate named entity mappings for the tokens
def viterbi(hmm, observation):
  labels = list(hmm['initial'].keys())
  initial = hmm['initial']
  lst = []
  BPTR = []

  for i in range(len(observation)):
    token = observation[i]
    if i == 0:
      probs = {}
      for label in labels:
        probs[label] = math.log(hmm['initial'][label]) + get_emission_smoothing(hmm['emission'], label, token)
      lst.append(probs)
    else:
      BPTR.append({})
      probs = {}
      for label in labels:
        max_lst = []
        for prev_label in labels:
          max_lst.append(
              lst[i-1][prev_label] + get_transition_smoothing(hmm['transition'], prev_label, label)
          )

        max_prob = max(max_lst)
        BPTR[i-1][label] = labels[max_lst.index(max_prob)]
        probs[label] = max_prob + get_emission_smoothing(hmm['emission'], label, token)
      lst.append(probs)

  final_probs = [lst[-1][key] for key in labels]
  max_prob = max(final_probs)
  backpointer = labels[final_probs.index(max_prob)]
  final_labels = [backpointer]

  BPTR.reverse()
  for dic in BPTR:
    backpointer = dic[backpointer]
    final_labels.append(backpointer)
  
  final_labels.reverse()
    
  # print(lst)
  # print(final_labels)
  return final_labels



In [ ]:
# here's a samplle observations that you can use to test your code
obs_1 = ['Cornell',
 'University',
 'is',
 'located',
 'in',
 'Germany',
 'and',
 'was',
 'founded',
 'by',
 'Polish',
 'schoolgirl']

obs_1 = ['Germany',
 'University',
 'is',
 'located',
 'in',
 'Ithaca',
 'and',
 'was',
 'founded',
 'by',
 'Ezra',
 'Cornell']

# obs_1 = "Bavarian lawmaker Markus Söder says conservatives in Germany will likely face a new era in opposition. His assessment contrasts strongly with that of fellow conservative leader Armin Laschet.".split()

viterbi(observation=obs_1, hmm=build_hmm(train['text'], train['NER']))

['LOC', 'LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

## **Validation Step**

---

In this part of the project, we expect you to split the training data into train and validation datasets. You may use whatever split you see fit and use any external libraries to perform this split. You may want to look into the following function for splitting data: [sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

Once you have split the data, train your HMM model on the training data and evaluate it on the validation data. Report **Entity Level Mean F1**, which was explained earlier. Please use the code we have provided below to compute this metric.

Please also take a look into your misclassified cases, as we will be performing error analysis in the *Evaluation* section. We expect smoothing, unknown word handling and correct emission (i.e., lexical generation) probabilities.

Consider the example below. After getting a sequence of NER labels for the sequence of tokens from your Viterbi algorithm implementation, you need to convert the sequence of tokens, associated token indices and NER labels into a format which can be used to calculate **Entity Level Mean F1**. We do this by finding the starting and ending indices of the spans representing each entity (as given in the corpus) and adding it to a list that is associated with the label with which the spans are labelled. To score your validation data on Google Colab or your local device, you can get a dictionary format as shown in the picture below from the function **format_output_labels** of both the predicted and true label sequences, and use the two dictionaries as input to the **mean_f1** function.

NOTE: We do **not** include the spans of the tokens labelled as "O" in the formatted dictionary output.

![picture](https://docs.google.com/uc?export=download&id=1M57DEHgfusVPU_hlvmiOpkS3yn9GGEgj)

In [ ]:
def format_output_labels(token_labels, token_indices):
    """
    Returns a dictionary that has the labels (LOC, ORG, MISC or PER) as the keys, 
    with the associated value being the list of entities predicted to be of that key label. 
    Each entity is specified by its starting and ending position indicated in [token_indices].

    Eg. if [token_labels] = ["ORG", "ORG", "O", "O", "ORG"]
           [token_indices] = [15, 16, 17, 18, 19]
        then dictionary returned is 
        {'LOC': [], 'MISC': [], 'ORG': [(15, 16), (19, 19)], 'PER': []}

    :parameter token_labels: A list of token labels (eg. PER, LOC, ORG or MISC).
    :type token_labels: List[String]
    :parameter token_indices: A list of token indices (taken from the dataset) 
                              corresponding to the labels in [token_labels].
    :type token_indices: List[int]
    """
    label_dict = {"LOC":[], "MISC":[], "ORG":[], "PER":[]}
    prev_label = token_labels[0]
    start = token_indices[0]
    for idx, label in enumerate(token_labels):
      if prev_label != label:
        end = token_indices[idx-1]
        if prev_label != "O":
            label_dict[prev_label].append((start, end))
        start = token_indices[idx]
      prev_label = label
      if idx == len(token_labels) - 1:
        if prev_label != "O":
            label_dict[prev_label].append((start, token_indices[idx]))
    return label_dict

In [ ]:
# Code for mean F1

import numpy as np

def mean_f1(y_pred_dict, y_true_dict):
    """ 
    Calculates the entity-level mean F1 score given the actual/true and 
    predicted span labels.
    :parameter y_pred_dict: A dictionary containing predicted labels as keys and the 
                            list of associated span labels as the corresponding
                            values.
    :type y_pred_dict: Dict<key [String] : value List[Tuple]>
    :parameter y_true_dict: A dictionary containing true labels as keys and the 
                            list of associated span labels as the corresponding
                            values.
    :type y_true_dict: Dict<key [String] : value List[Tuple]>

    Implementation modified from original by author @shonenkov at
    https://www.kaggle.com/shonenkov/competition-metrics.
    """
    F1_lst = []
    for key in y_true_dict:
        TP, FN, FP = 0, 0, 0
        num_correct, num_true = 0, 0
        preds = y_pred_dict[key]
        trues = y_true_dict[key]
        for true in trues:
            num_true += 1
            if true in preds:
                num_correct += 1
            else:
                continue
        num_pred = len(preds)
        if num_true != 0:
            if num_pred != 0 and num_correct != 0:
                R = num_correct / num_true
                P = num_correct / num_pred
                F1 = 2*P*R / (P + R)
            else:
                F1 = 0      # either no predictions or no correct predictions
        else:
            continue
        F1_lst.append(F1)
    return np.mean(F1_lst)

In [ ]:
# Usage using above example

pred_token_labels = ["ORG", "O", "PER", "PER", "O", "LOC", "O", "O", "O", "O", "MISC", "O", "O", "O", "O", "LOC"]
true_token_labels = ["ORG", "O", "PER", "PER", "O", "LOC", "O", "O", "O", "O", "MISC", "MISC", "O", "O", "O", "LOC"]
token_indices = [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]

y_pred_dict = format_output_labels(pred_token_labels, token_indices)
print("y_pred_dict is : " + str(y_pred_dict))
y_true_dict = format_output_labels(true_token_labels, token_indices)
print("y_true_dict is : " + str(y_true_dict))

print("Entity Level Mean F1 score is : " + str(mean_f1(y_pred_dict, y_true_dict)))

y_pred_dict is : {'LOC': [(18, 18), (28, 28)], 'MISC': [(23, 23)], 'ORG': [(13, 13)], 'PER': [(15, 16)]}
y_true_dict is : {'LOC': [(18, 18), (28, 28)], 'MISC': [(23, 24)], 'ORG': [(13, 13)], 'PER': [(15, 16)]}
Entity Level Mean F1 score is : 0.75


In [ ]:
# Evaluate/validate your model here
from sklearn.model_selection import train_test_split

text_train, text_val, NER_train, NER_val, index_train, index_val = train_test_split(train['text'], train['NER'], train['index'], test_size=0.2)
# print(len(text_train), len(text_val))

# print(text_train)
# print(text_val)
# print(NER_train)
# print(NER_val)
# print(index_train)
# print(index_val)

In [ ]:
hmm=build_hmm(text_train, NER_train)
predicted_labels = []
for i in range(len(text_val)):
  val_example = text_val[i]
  labels = viterbi(hmm, val_example)
  predicted_labels.append(labels)

f1_lst = []
for i in range(len(NER_val)):
  pred_token_labels = predicted_labels[i]
  token_indices = index_val[i]
  true_token_labels = NER_val[i]

  y_pred_dict = format_output_labels(pred_token_labels, token_indices)
  # print("y_pred_dict is : " + str(y_pred_dict))
  y_true_dict = format_output_labels(true_token_labels, token_indices)
  # print("y_true_dict is : " + str(y_true_dict))
  f1 = mean_f1(y_pred_dict, y_true_dict)
  # print("Entity Level Mean F1 score is : " + str(f1))
  f1_lst.append(f1)

print("Mean f1 : " + str(sum(f1_lst)/len(f1_lst)))



Mean f1 : 0.5917124620211327


### **Q2.1: Explain your HMM Implementations**

Explain how you implemented the HMM including the Viterbi algorithm (e.g. **which algorithms/data structures** you used). Make clear which parts were implemented from scratch vs. obtained via an existing package. Explain and motivate any design choices providing the intuition behind them (e.g. which methods you used for your HMM implementation, and why?). (Please answer on the written questions template document)


### **Q2.2: Results Analysis**

Explain here how you evaluated the models. Summarize the performance of your system and any variations that you experimented with on the validation datasets. Put the results into clearly labeled tables or diagrams and include your observations and analysis. (Please answer on the written questions template document)


### **Q2.3: Error Analysis**
When did the system work well? When did it fail?  Any ideas as to why? How might you improve the system? (Please answer on the written questions template document)


### **Q2.4: What is the effect of unknown word handling and smoothing?**
(Please answer on the written questions template document)

# **Part 3: Maximum Entropy Markov Model** 💫

---

In this section, you will implement a Maximum Entropy Markov Model (**MEMM**) to perform the same NER task. Your model should consist of a MaxEnt classifier with Viterbi decoding. 

1. We have already performed tokenizations for documents. You can either use a MaxEnt classifier from an existing package or write the MaxEnt code yourself. **Important note:  MaxEnt classifiers are statistically equivalent to multi-class logistic regression, so you can use packages for multi-class LR instead of MaxEnt.**

2. Use the classifier to learn a probability $P(t_i|features)$. You may replace either the lexical generation probability – $P(w_i|t_i)$ – or the transition probability – $P(t_i|t_{i−1})$ – in the HMM with it, or you may replace the entire *lexical generation probability * transition probability*  calculation – $P (w_i|t_i) ∗ P (t_i|t_{i−1)} – $ in the HMM with it. 

3. To train such classifier, you need to pick some feature set. The content of the feature set is up to your choice. You should be trying different feature sets, and evaluate your choices on the validation set. Pick the feature set that performs overall the best according to the F1 measure. If you draw inspiration for your features from external sources, please link them in the code.

4. Use your own implementation of the **Viterbi algorithm**, which you can modify from the one you developed for the HMM model. You will need the probabilities that you obtain from the MaxEnt classifier. 

5. Remember to use same training and validation split when evaluating the MEMM to have a **fair comparison** with your **HMM model**.


Please also take a look into your misclassified cases, as we will be performing error analysis in *Evaluation* section. 





---
Here's a summary of the workflow for Part 3:

![alt text](https://drive.google.com/uc?export=view&id=14VfjW3yDyXLojWM_u0LeJYdDOSLkElBn)




Note that we have not provided any skeleton code for how you should do feature engineering since this is meant to be an open ended task and we want you to experiment with the dataset. However, please remember to make sure that you code is concise, clean, and readable! Ultimately, we expect a function or class  mapping a sequence of tokens to a sequence of labels. At the end of this section you should have done the following:
1. Extract Features
2. Build & Train MaxEnt
3. Call Viterbi when evaluating

### **Feature Engineering**
---

In [ ]:
from sklearn.linear_model import LogisticRegression
import pandas as pd

# pd.get_dummies(train['POS'][0])
# COLUMNS_RESET = True
CLASSES = ['O', 'LOC', 'ORG', 'PER', 'MISC']
COLUMNS = ['"', '$', "''", '(', ')', ',', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW',
           'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS',
           'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP',
           'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT',
           'WP', 'WP$', 'WRB']


def flatten(tokens, POSs, NERs, indices):
  POS_lst = []
  token_lst = []
  NER_lst = []
  indices_lst = []
  for i in range(len(tokens)):
    POS_lst.extend(POSs[i])
    token_lst.extend(tokens[i])
    NER_lst.extend([CLASSES.index(n) for n in NERs[i]])
    indices_lst.extend(indices[i])
  return token_lst, POS_lst, NER_lst, indices_lst

# POS_lst = []
# token_lst = []
# NER_lst = []
# for i in range(len(train['POS'])):
#   POS_lst.extend(train['POS'][i])
#   token_lst.extend(train['text'][i])
#   NER_lst.extend([CLASSES.index(n) for n in train['NER'][i]])


# token_lst, POS_lst, NER_lst, indices_lst = flatten(train['text'], train['POS'], train['NER'], train['index'])

# print(NER_lst)
# print(token_lst)
 
def feature_extraction(POS_lst):
  global COLUMNS_RESET
  features = pd.get_dummies(POS_lst)

  # print(list(features.columns))
  # if COLUMNS_RESET:
  #   COLUMNS = list(features.columns)
  #   COLUMNS_RESET = False

  for i in COLUMNS:
    if i not in features.columns:
      features[i] = 0

  features = features[COLUMNS]
  features_copy = features.copy()

  # Add previous word POS
  temp_features = features.drop(features_copy.shape[0]-1)

  temp_features.loc[-1, :] = [0] * len(COLUMNS)
  temp_features = temp_features.sort_index()

  for i in COLUMNS:
    col_name = "Prev_" + i
    features[col_name] = list(temp_features[i])

  # Add next word POS
  temp_features = features_copy.drop(0)

  temp_features.loc[temp_features.shape[0]+1, :] = [0] * len(COLUMNS)
  temp_features = temp_features.sort_index()

  for i in COLUMNS:
    col_name = "Post_" + i
    # print( len(list(temp_features[i])))
    # print(len(features[i]))
    features[col_name] = list(temp_features[i])
  # print(features)
  return features

def build_lrm(features, expected):
  lrm = LogisticRegression(solver='newton-cg')
  lrm.fit(features, expected)
  return lrm
  
# features = feature_extraction(POS_lst)
# lrm = build_lrm(features, NER_lst)

# predicted = lrm.predict(features)
# print(NER_lst)
# print(list(predicted))
# print(lrm.score(features, NER_lst))

### **MEMM Implementation**
---

In [ ]:
# Your implementation here

# takes in the hmm build above and an observation: list of tokens
# and returns the appropriate named entity mappings for the tokens
def MEMM(hmm, lrm, observation, observation_POS):

  test_features = feature_extraction(observation_POS)

  # print(test_features)
  prediction = lrm.predict_proba(test_features)
  
  labels = list(hmm['initial'].keys())
  initial = hmm['initial']
  lst = []
  BPTR = []

  for i in range(len(observation)):
    token = observation[i]
    if i == 0:
      probs = {}
      for label in labels:
        probs[label] = math.log(hmm['initial'][label]) + math.log(prediction[0][CLASSES.index(label)]) + get_emission_smoothing(hmm['emission'], label, token)
      lst.append(probs)
    else:
      BPTR.append({})
      probs = {}
      for label in labels:
        max_lst = []
        for prev_label in labels:
          max_lst.append(
              lst[i-1][prev_label]#] + get_transition_smoothing(hmm['transition'], prev_label, label)
          )

        max_prob = max(max_lst)
        BPTR[i-1][label] = labels[max_lst.index(max_prob)]
        # print(prediction[i][CLASSES.index(label)])
        probs[label] = max_prob + math.log(prediction[i][CLASSES.index(label)]) + get_emission_smoothing(hmm['emission'], label, token)
      lst.append(probs)

  final_probs = [lst[-1][key] for key in labels]
  max_prob = max(final_probs)
  backpointer = labels[final_probs.index(max_prob)]
  final_labels = [backpointer]

  BPTR.reverse()
  for dic in BPTR:
    backpointer = dic[backpointer]
    final_labels.append(backpointer)
  
  final_labels.reverse()
    
  # print(lst)
  # print(final_labels)
  return final_labels

### **Validation**
---
In this section we want you to run your MaxEnt model on the validation dataset you extracted earlier. We want you to play around with different combinations of features in order to find which features work the best for your implementation. You will be asked to write about this process in detail in written question 3.3 so please spend time experimenting with features! Once again, please use the code we provided for computing Entity Level Avg F1 earlier when validating your model.

In [ ]:
from sklearn.model_selection import train_test_split

text_train, text_val, NER_train, NER_val, index_train, index_val, POS_train, POS_val =\
 train_test_split(train['text'], train['NER'], train['index'], train['POS'], test_size=0.01)

In [ ]:
# Run your model on validation set
# You will need to 
# 1. Call your function above to get a prediction result on Validation Set
# 2. Report Metrics
# (See if you need to modify your feature set)

token_lst, POS_lst, NER_lst, indices_lst = flatten(text_train, POS_train, NER_train, index_train)
features = feature_extraction(POS_lst)
# features = feature_extraction(token_lst)


# print(features.columns)
lrm = build_lrm(features, NER_lst)

hmm=build_hmm(text_train, NER_train)

predicted_labels = []
for i in range(len(text_val)):
  val_example = text_val[i]
  POS_example = POS_val[i]
  # print(POS_example)
  labels = MEMM(hmm, lrm, val_example, POS_example)
  predicted_labels.append(labels)

f1_lst = []
for i in range(len(NER_val)):
  pred_token_labels = predicted_labels[i]
  token_indices = index_val[i]
  true_token_labels = NER_val[i]

  y_pred_dict = format_output_labels(pred_token_labels, token_indices)
  # print("y_pred_dict is : " + str(y_pred_dict))
  y_true_dict = format_output_labels(true_token_labels, token_indices)
  # print("y_true_dict is : " + str(y_true_dict))
  f1 = mean_f1(y_pred_dict, y_true_dict)
  # print("Entity Level Mean F1 score is : " + str(f1))
  f1_lst.append(f1)

print("Mean f1 : " + str(sum(f1_lst)/len(f1_lst)))


Mean f1 : 0.7016762126137126


In [ ]:
token_lst, POS_lst, NER_lst, indices_lst = flatten(text_train, POS_train, NER_train, index_train)
features = feature_extraction(POS_lst)
# features = feature_extraction(token_lst)


# print(features.columns)
lrm = build_lrm(features, NER_lst)

hmm = build_hmm(text_train, NER_train)

In [ ]:
# Error Analysis
# 20, 22, 
ind = 1
obs_2 = text_val[ind][6:15]
obs_2_POS = POS_val[ind][6:15]
obs_2_NER = NER_val[ind][6:15]

# obs_1 = "Bavarian lawmaker Markus Söder says conservatives in Germany will likely face a new era in opposition. His assessment contrasts strongly with that of fellow conservative leader Armin Laschet.".split()
# print(obs_2)
# print("MEMM", MEMM(hmm, lrm, obs_2, obs_2_POS))
# print("Vite", viterbi(hmm, obs_2))
# print("True", obs_2_NER)

comparison = [MEMM(hmm, lrm, obs_2, obs_2_POS), viterbi(hmm, obs_2), obs_2_NER]
df = pd.DataFrame(comparison)
df.index = ["MEMM", "HMM", "Real"]
df.columns = obs_2
df

,silence,.,Kieran,Murray,LAS,CRUCES,",",N.M.,1996-08-22
MEMM,O,O,PER,PER,ORG,PER,O,PER,O
HMM,O,O,PER,PER,PER,PER,O,O,O
Real,O,O,PER,PER,LOC,LOC,O,O,O


In [ ]:
#Comparing our feature set to the training data
counts_of_POS ={}

for j in range(len(train["POS"])):
  for i in range(len(train['POS'][j])):
      
    POS_at_i = train['POS'][j][i]
    NER_at_i = train['NER'][j][i]

    if POS_at_i not in counts_of_POS:
      counts_of_POS[POS_at_i] = {}
    if NER_at_i not in counts_of_POS[POS_at_i]:
      counts_of_POS[POS_at_i][NER_at_i] = 1
    else: 
      counts_of_POS[POS_at_i][NER_at_i] += 1

maxOfEach = {}
percentOfTotalTags=[]
for POS in counts_of_POS:
  maxNER = 0
  nameNER =""

  totalTags = 0
  for NER in counts_of_POS[POS]:
    totalTags += counts_of_POS[POS][NER]
    if counts_of_POS[POS][NER] > maxNER:
      maxNER = counts_of_POS[POS][NER]
      nameNER = NER
  maxOfEach[POS] ={}
  maxOfEach[POS][nameNER] = maxNER
  percentOfTotalTags.append(maxNER/totalTags)

keysofMax = list(maxOfEach.keys())
NERlst = []
totalofMAX = []

for POS in maxOfEach.keys():
  for NER in maxOfEach[POS]:
    NERlst.append(NER)
    totalofMAX.append(maxOfEach[POS][NER])
for i in range(len(keysofMax)):
  keysofMax[i] = keysofMax[i] + " : " + NERlst[i]

POSandNERdf = pd.DataFrame([totalofMAX, percentOfTotalTags])

#POSandNERdf = pd.DataFrame(totalofMAX)
POSandNERdf.index = ["Max NER for given POS token", "Percentage of Total NER Tags"]
POSandNERdf.columns = keysofMax
POSandNERdf
#set like list
# for elt in counts_of_POS.keys():
#   counts_of_POS[elt]= set(counts_of_POS[elt])
# print(counts_of_POS)

,NNP : PER,NN : O,NNS : O,IN : O,. : O,RB : O,CD : O,POS : O,VBD : O,TO : O,DT : O,JJ : O,VBN : O,", : O",CC : O,VB : O,VBG : O,VBZ : O,PRP : O,JJR : O,WDT : O,VBP : O,RP : O,: : O,""" : O",( : O,$ : O,) : O,PRP$ : O,MD : O,NNPS : ORG,JJS : O,LS : O,WP : O,RBR : O,FW : O,EX : O,WRB : O,SYM : O,WP$ : O,PDT : O,UH : O,'' : O,RBS : O,NN|SYM : ORG
Max NER for given POS token,8370.000000,18380.000000,7707.000000,15078.000000,5924.000000,3106.000000,15038.000000,1177.000000,6640.000000,2771.000000,10681.000000,7869.000000,3292.000000,5808.000000,2894.000000,3295.000000,1966.000000,1915.00000,2508.000000,277.000000,416.000000,1158.000000,416.000000,1926.000000,1701.0,2348.000000,298.000000,2351.000000,1194.0,960.000000,227.000000,178.000000,11.0,413.0,132.0,95.000000,107.0,311.0,343.00,17.0,26.0,10.00000,30.0000,29.000000,2.000000
Percentage of Total NER Tags,0.305664,0.964677,0.963134,0.988851,0.998483,0.973973,0.993919,0.970322,0.997147,0.988936,0.993304,0.834907,0.987995,0.998281,0.979689,0.962606,0.957623,0.98407,0.996424,0.926421,0.976526,0.993139,0.976526,0.997927,1.0,0.994915,0.856322,0.994922,1.0,0.997921,0.424299,0.936842,1.0,1.0,1.0,0.612903,1.0,1.0,0.98,1.0,1.0,0.47619,0.9375,0.966667,0.666667


### **Q3.1: Implementation Details**
Explain how you implemented the MEMM and whether/how you modified Viterbi (e.g. which algorithms/data structures you used, what features are included). Make clear which parts were implemented from scratch vs. obtained via an existing package. (Please answer on the written questions template document)



### **Q3.2: Results Analysis**
Explain here how you evaluated the MEMM model. Summarize the performance of your system and any variations that you experimented with the validation datasets. Put the results into clearly labeled tables or diagrams and include your observations and analysis. (Please answer on the written questions template document)



### **Q3.3: Feature Engineering**
What features are considered most important by your MaxEnt Classifier? Why do you think these features make sense? Describe your experiments with feature sets. An analysis on feature selection for the MEMM is required – e.g. what features **help most**, why? An **error analysis** is required – e.g. what sorts of errors occurred, why? (Please answer on the written questions template document)

### **Q3.4: Room for Improvement**
When did the system work well, when did it fail and any ideas as to why? How might you improve the system?


# **Part 4: Comparing HMMs and MEMMs**

---

In this section you will be asked to analyze and compare the models you have developed!




### **Q4.1: Result Comparison**
Compare here your results (validation scores) for your HMM and the MEMM. Which of them performs better? Why? (Please answer on the written questions template document)

### **Q4.2: Error Analysis 1**
Do some error analysis. What are error patterns you observed that the HMM makes but the MEMM does not? Try to justify why/why not? **Please give examples from the dataset.** (Please answer on the written questions template document)

### **Q4.3: Error Analysis 2**
What are error patterns you observed that MEMM makes but the HMM does not? Try to justify what you observe? **Please give examples from the dataset.** (Please answer on the written questions template document)

# **Part 5: Kaggle Submission**
---

Using the best-performing system from among all of your HMM and MEMM models, generate predictions for the test set, and submit them to Kaggle at https://www.kaggle.com/c/cs4740-fa21-p2. Note, you **need** to use our tokenizer as the labels on Kaggle corresponds to these. Below, we provide a function that submits given predicted tokens and associated token indices in the correct format. As a scoring metric on Kaggle, we use **Entity Level Mean F1**.

Your submission to Kaggle should be a CSV file consisting of five lines and two columns. The first line is a fixed header, and each of the remaining four lines corresponds to one of the four types of named entities. The first column is the label identifier *Id* (one of PER, LOC, ORG or MISC), and the second column *Predicted* is a list of entities (separated by single space) that you predict to be of that type. Each entity is specified by its starting and ending index (concatenated by a hypen) as given in the test corpus. 

You can use the function **create_submission** that takes the list of predicted labels and the list of associated token indices as inputs and creates the the output CSV file at a specified path.

NOTE: Ensure that there are **no** rows with *Id* = "O" in your Kaggle Submission.

![picture](https://docs.google.com/uc?export=download&id=1pQkAyOdWQz62jB-YBaj8mHuwI6iWJ1GZ)

In [ ]:
import csv

def create_submission(output_filepath, token_labels, token_inds):
    """
    :parameter output_filepath: The full path (including file name) of the output file, 
                                with extension .csv
    :type output_filepath: [String]
    :parameter token_labels: A list of token labels (eg. PER, LOC, ORG or MISC).
    :type token_labels: List[String]
    :parameter token_indices: A list of token indices (taken from the dataset) 
                              corresponding to the labels in [token_labels].
    :type token_indices: List[int]
    """
    label_dict = format_output_labels(token_labels, token_inds)
    with open(output_filepath, mode='w') as csv_file:
        fieldnames = ['Id', 'Expected']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for key in label_dict:
            p_string = " ".join([str(start)+"-"+str(end) for start,end in label_dict[key]])
            writer.writerow({'Id': key, 'Expected': p_string})

In [ ]:
hmm=build_hmm(train['text'], train['NER'])
token_labels = []
token_inds = []

for obs_id in range(len(test['text'])):
  obs = test['text'][obs_id]
  token_labels.extend(viterbi(observation=obs, hmm=hmm))
  token_inds.extend(test['index'][obs_id])
# print(token_inds)
# print(token_labels)

file_path = os.path.join(path,'submit.csv')
create_submission(file_path, token_labels, token_inds)

In [ ]:
token_lst, POS_lst, NER_lst, indices_lst = flatten(train['text'], train['POS'], train['NER'], train['index'])
features = feature_extraction(POS_lst)
# features = feature_extraction(token_lst)


# print(features.columns)
lrm = build_lrm(features, NER_lst)

hmm = build_hmm(text_train, NER_train)

token_labels = []
token_inds = []

for obs_id in range(len(test['text'])):
  obs = test['text'][obs_id]
  pos = test['POS'][obs_id]
  token_labels.extend(MEMM(hmm, lrm, obs, pos))
  token_inds.extend(test['index'][obs_id])
# print(token_inds)
# print(token_labels)

file_path = os.path.join(path,'submit_MEMM.csv')
create_submission(file_path, token_labels, token_inds)

---
### **Q5.1: Competition Score**

Include your **team name** and the **screenshot** of your best score from Kaggle. (Please answer on the written questions template document)

In [ ]:
#Convert into PDF
%%capture
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc


In [ ]:
%%capture
# the red text is a placeholder! Change it to your directory structure!
!cp 'drive/My Drive/CS 4740/Project 2/CS_4740_FA21_p2_nar73_krm74.ipynb' ./ 

In [ ]:
# the red text is a placeholder! Change it to the name of this notebook!
!jupyter nbconvert --to PDF "CS_4740_FA21_p2_nar73_krm74.ipynb"

[NbConvertApp] Converting notebook CS_4740_FA21_p2_nar73_krm74.ipynb to PDF
[NbConvertApp] Writing 130901 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: [u'xelatex', u'./notebook.tex', '-quiet']
[NbConvertApp] CRITICAL | xelatex failed: [u'xelatex', u'./notebook.tex', '-quiet']
This is XeTeX, Version 3.14159265-2.6-0.99998 (TeX Live 2017/Debian) (preloaded format=xelatex)
 restricted \write18 enabled.
entering extended mode
(./notebook.tex
LaTeX2e <2017-04-15>
Babel <3.18> and hyphenation patterns for 3 language(s) loaded.
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2014/09/29 v1.4h Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size11.clo))
(/usr/share/texlive/texmf-dist/tex/latex/tcolorbox/tcolorbox.sty
(/usr/share/texlive/texmf-dist/tex/latex/pgf/basiclayer/pgf.sty
(/usr/share/texlive/texmf-dist/tex/latex/pgf/utilities/pgfrcs.sty
(/usr/share/texlive/texmf-dist/tex/generic/